In [1]:
# Donchian weekly trend following system.
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [2]:
# Function definitions.
def donchian(prices, period):
    '''Calculate upper, lower, & middle Donchian lines.'''
    df = pd.DataFrame(prices.copy())
    df['upr'] = df.high.rolling(period).max().shift(periods=1)
    df['lwr'] = df.low.rolling(period).min().shift(periods=1)
    df['mid'] = 0.5 * (df.upr + df.lwr)
    return df


def plot_profit(trade_list):
    """Plot cumulative profit vs date."""
    
    # Create plot series.
    ts = td.cum_profit
    
    # Create plot trace.
    trace = go.Scatter(
        x=ts.index,
        y=ts.values,
        line=dict(color='rgba(92, 230, 174, 1.0)', width=1.0),
        yhoverformat=".0f",
    )

    # Create figure and add trace.
    fig = go.Figure(trace)

    # Customise plot.
    fig.update_layout(
        autosize=True,
        showlegend=False,
        paper_bgcolor='rgba(28, 28, 28, 1.0)',
        plot_bgcolor='rgba(28, 28, 28, 1.0)',
        font=dict(color='rgba(226, 226, 226, 1.0)'),
        title=dict(
            text=f'{exchange} {tidm}'
            + f'<br>{strategy} {timeframe}',
            font_color='rgba(226, 226, 226, 1.0)',
            font_size=16,
        ),
        xaxis=dict(rangeslider=dict(visible=False)),
        hovermode='x unified',
        hoverlabel=dict(bgcolor='rgba(28, 28, 28, 0.5)'),
        dragmode='pan',
    )

    fig.update_xaxes(
        linecolor='rgba(226, 226, 226, 1.0)',
        gridcolor='rgba(119, 119, 119, 0.5)',
        mirror=True,
        title=f'Position Size = ' + '{:,}'.format(position_size),
    )

    fig.update_yaxes(
        linecolor='rgba(226, 226, 226, 1.0)',
        gridcolor='rgba(119, 119, 119, 0.5)',
        mirror=True,
        side='right',
        title=f'Cumulative Profit',
    )

    # Display plot.
    fig.show(
        config={
            'scrollZoom': True,
            'modeBarButtonsToRemove': ['zoom', 'select2d', 'lasso2d'],
        }
    )
    return


def style_trade(trade_list):
    '''Format trade list.'''
    return (
        trade_list.style
            .format({
                'date': '{:%Y-%m-%d}',
                'open': '{:.3f}',
                'high': '{:.3f}',
                'low': '{:.3f}',
                'close': '{:.3f}',
                'upr': '{:.3f}',
                'lwr': '{:.3f}',
                'mid': '{:.3f}',
                'volatility': '{:.1%}',
                'risk_raw': '{:,.2f}',
                'risk': '{:,.2f}',
                'charges': '{:,.2f}',
                'sduty': '{:,.2f}',
                'cost': '{:,.2f}',
                'value': '{:,.2f}',
                'profit': '{:,.2f}',
                'pct': '{:,.1%}',
                'annual': '{:,.1%}',
                'cum_profit': '{:,.2f}',
            })
    )


def weekly(exchange, tidm):
    '''Generate weekly prices from SharePad csv file of daily prices.'''
    df = pd.read_csv(
        f'{exchange}_{tidm}_prices.csv',
        header=0,
        names=['date', 'open', 'high', 'low', 'close'],
        index_col=0,
        usecols=[0, 1, 2, 3, 4],
        parse_dates=True,
        dayfirst=True,
    )
    df = df.sort_index()
    functions = dict(open='first', high='max', low='min', close='last')
    df = df.resample('W-FRI').agg(functions)
    df = df / 100
    df.insert(0, 'tidm', tidm)
    return df

In [3]:
# Trade parameters.
exchange = 'LSE'
tidm = 'HSV'
timeframe = 'Weekly'
strategy = 'Donchian'
period = 48  # System look back period.
position_size = 1875  # Position size in major currency unit.
risk_pct = 0.2  # Percentage risk per trade.
commission = 11.95  # Commission per trade.
sduty = 0.5  # Stamp Duty percentage.

In [4]:
# Import weekly prices.
prices = weekly(exchange, tidm)

# Donchian channel.
dc = donchian(prices, period)
dc['period'] = period

# Raw entry & exit signals.
dc['entry'] = np.where(dc.close > dc.upr, 1, 0)
dc['exit'] = np.where(dc.close < dc.mid, 1, 0)

# State variable.
dc['state'] = 0
for i in range(period, len(dc)):
    if dc.loc[dc.index[i], 'entry'] == 1 and dc.loc[dc.index[i - 1], 'state'] == 0:
        dc.loc[dc.index[i], 'state'] = 1
    elif dc.loc[dc.index[i], 'exit'] == 1:
        dc.loc[dc.index[i], 'state'] = 0
    else:
        dc.loc[dc.index[i], 'state'] = dc.loc[dc.index[i - 1], 'state']

# Buy & sell signals.
dc['buy'] = np.where(np.logical_and(dc.state == 1, dc.state.shift(periods=1) == 0), 1, 0)
dc['sell'] = np.where(np.logical_and(dc.state == 0, dc.state.shift(periods=1) == 1), 1, 0)

# Filter signals.
dc = pd.concat([dc[dc.buy == 1] , dc[dc.sell == 1]], axis=0)

# Signals indexed by date.
dc = dc.sort_index()

In [5]:
# Position size (buy side).
dc['volatility'] = np.where(dc.buy == 1, abs((dc.mid - dc.close) / dc.close), 0)
dc['risk'] = np.where(dc.buy == 1, ((position_size * risk_pct) / dc.volatility), 0)
dc['shares'] = np.where(dc.buy == 1, (dc.risk / dc.close).astype('int'), 0)

# Position size (sell side).
for index, row in dc.iterrows():
    if row['buy'] == 1:
        shares = row['shares']
    else:
        dc.at[index, 'shares'] = shares

# Charges.
dc['charges'] = dc.index.values
dc.charges = dc.charges.shift()
dc.charges = np.where(dc.index == dc.charges.values, 0, commission)

# Stamp duty.
dc['sduty'] = np.where(dc.buy==1, ((sduty / 100) * dc.close * dc.shares), 0)

# Cost to buy.
dc['cost'] = np.where(dc.buy == 1, (dc.risk + dc.charges + dc.sduty), 0)

# Cost to sell.
for index, row in dc.iterrows():
    if row['buy'] == 1:
        cost = row['cost']
    else:
        dc.at[index, 'cost'] = cost
        
# Returns.
dc['value'] = np.where(dc.sell == 1, ((dc.close * dc.shares) - dc.charges), 0)
dc['profit'] = np.where(dc.sell == 1, dc.value - dc.cost, 0)
dc['pct'] = np.where(dc.sell == 1, (dc.profit / dc.cost), 0)

# Trade duration.
dc['days'] = 0
for index, row in dc.iterrows():
    if row['entry'] == 1:
        start_date = index
    else:
        dc.at[index, 'days'] = index - start_date
dc.days = dc.days.astype('timedelta64[D]')
dc.days = dc.days.dt.days

# Annual percentage return.
dc['annual'] = ((np.power(1 + dc.profit / dc.cost, (365 / dc.days)) - 1))

# Cumulative profit.
# dc['cum_profit'] = dc.profit.cumsum()

# Signals indexed by integer.
dc = dc.reset_index()
style_trade(dc)

,date,tidm,open,high,low,close,upr,lwr,mid,period,entry,exit,state,buy,sell,volatility,risk,shares,charges,sduty,cost,value,profit,pct,days,annual
0,2001-12-07,HSV,1.195,1.228,1.195,1.228,1.206,0.878,1.042,48,1,0,1,1,0,15.1%,"2,478.47",2018,11.95,12.39,"2,502.80",0.00,0.00,0.0%,0,0.0%
1,2002-09-06,HSV,1.179,1.185,1.141,1.147,1.359,0.991,1.175,48,0,1,0,0,1,0.0%,0.00,2018,11.95,0.00,"2,502.80","2,302.49",-200.31,-8.0%,273,-10.6%
2,2003-09-05,HSV,1.125,1.174,1.120,1.171,1.141,0.786,0.964,48,1,0,1,1,0,17.7%,"2,122.96",1813,11.95,10.61,"2,145.52",0.00,0.00,0.0%,0,0.0%
3,2007-07-27,HSV,3.877,3.941,3.567,3.597,4.392,3.416,3.904,48,0,1,0,0,1,0.0%,0.00,1813,11.95,0.00,"2,145.52","6,509.23","4,363.71",203.4%,1421,33.0%
4,2009-09-11,HSV,3.190,3.440,3.168,3.351,3.300,1.779,2.539,48,1,0,1,1,0,24.2%,"1,547.75",461,11.95,7.72,"1,567.43",0.00,0.00,0.0%,0,0.0%
5,2010-11-12,HSV,4.699,4.715,4.375,4.394,5.406,3.416,4.411,48,0,1,0,0,1,0.0%,0.00,461,11.95,0.00,"1,567.43","2,013.59",446.16,28.5%,427,23.9%
6,2011-05-06,HSV,5.290,5.449,5.283,5.428,5.406,4.211,4.809,48,1,0,1,1,0,11.4%,"3,287.12",605,11.95,16.42,"3,315.49",0.00,0.00,0.0%,0,0.0%
7,2011-08-05,HSV,5.223,5.317,4.681,4.862,5.761,4.366,5.064,48,0,1,0,0,1,0.0%,0.00,605,11.95,0.00,"3,315.49","2,929.74",-385.75,-11.6%,91,-39.1%
8,2013-05-24,HSV,2.384,2.857,2.384,2.746,2.709,1.563,2.136,48,1,0,1,1,0,22.2%,"1,686.44",614,11.95,8.43,"1,706.82",0.00,0.00,0.0%,0,0.0%
9,2013-10-18,HSV,2.720,2.751,2.423,2.531,3.206,1.991,2.599,48,0,1,0,0,1,0.0%,0.00,614,11.95,0.00,"1,706.82","1,541.96",-164.86,-9.7%,147,-22.3%


In [6]:
# Trade list.
td = dc[dc.sell == 1].copy()
td = td.drop(columns=['date', 'tidm','open', 'high', 'low', 'upr', 'lwr', 'mid','entry','exit', 'state', 'buy', 'sell'])
td = td.rename(columns={"close": "exit_price"})
td.insert(0, 'tidm', dc.tidm)
td.insert(1, 'entry_date', dc.date.shift())
td.insert(2, 'entry_price', dc.close.shift())
td.insert(3, 'exit_date', dc.date)
td.volatility = dc.volatility[dc.buy == 1]
td

,tidm,entry_date,entry_price,exit_date,exit_price,period,volatility,risk,shares,charges,sduty,cost,value,profit,pct,days,annual
1,HSV,2001-12-07,1.2277,2002-09-06,1.1469,48,NaN,0.0,2018,11.95,0.0,2502.803752,2302.4942,-200.309552,-0.080034,273,-0.105536
3,HSV,2003-09-05,1.1706,2007-07-27,3.5969,48,NaN,0.0,1813,11.95,0.0,2145.521228,6509.2297,4363.708472,2.033869,1421,0.329861
5,HSV,2009-09-11,3.3514,2010-11-12,4.3938,48,NaN,0.0,461,11.95,0.0,1567.427440,2013.5918,446.164360,0.284648,427,0.238764
7,HSV,2011-05-06,5.4277,2011-08-05,4.8623,48,NaN,0.0,605,11.95,0.0,3315.493308,2929.7415,-385.751808,-0.116348,91,-0.391116
9,HSV,2013-05-24,2.7462,2013-10-18,2.5308,48,NaN,0.0,614,11.95,0.0,1706.821512,1541.9612,-164.860312,-0.096589,147,-0.222924
11,HSV,2014-01-31,3.5280,2015-10-02,4.0760,48,NaN,0.0,434,11.95,0.0,1552.363682,1757.0340,204.670318,0.131844,609,0.077052
13,HSV,2016-05-27,4.8800,2018-03-02,7.0500,48,NaN,0.0,535,11.95,0.0,2636.864415,3759.8000,1122.935585,0.425860,644,0.222718
15,HSV,2018-05-25,8.9200,2018-12-14,8.8000,48,NaN,0.0,326,11.95,0.0,2941.522280,2856.8500,-84.672280,-0.028785,203,-0.051161
17,HSV,2019-04-05,10.9500,2020-03-13,9.4850,48,NaN,0.0,189,11.95,0.0,2093.546174,1780.7150,-312.831174,-0.149426,343,-0.158210


In [8]:
dc.volatility[dc.buy == 1]

0     0.151303
2     0.176640
4     0.242287
6     0.114081
8     0.222362
10    0.244657
12    0.143576
14    0.128643
16    0.181050
18    0.250234
Name: volatility, dtype: float64